In [ ]:
import os
from src.snslib.experiment.utils.parsing import parse_recording
from src.snslib.core.subject import TorchNetworkSubject
from src.snslib.core.utils.io_ import read_json
from src.snslib.core.utils.io_ import store_pickle
from src.snslib.experiment.utils.args import NATURAL_RECORDINGS


organization = read_json('path/to/your/nat_stats_fp.json')

import numpy as np

# Create a new dictionary with the same structure as 'organization'
data_dict = {}

for model, variants in organization.items():
    data_dict[model] = {}
    for variant, recordings in variants.items():
        data_dict[model][variant] = {}
        for recording_name, path in recordings.items():
            data_dict[model][variant][recording_name] = {}
            data_dict[model][variant][recording_name]['data'] = np.load(os.path.join(path, 'recordings.npy'))
            prms = read_json(os.path.join(path, 'params.json'))
            layer_info = TorchNetworkSubject(
            network_name=str(prms['net_name'])
            ).layer_info
            record_target = parse_recording(input_str=prms['rec_layers'], net_info=layer_info)
            #NOTE: labels are good in  the None scenario only if the recorded layer is linear
            idxs = list(record_target.values())[0]
            data_dict[model][variant][recording_name]['labels'] = idxs if idxs is not None else np.arange(data_dict[model][variant][recording_name]['data'].shape[0])
            
store_pickle(data_dict, NATURAL_RECORDINGS)